In [74]:
import pandas as pd

order_df = pd.read_csv("Company X - Order Report.csv")
pincode_df = pd.read_csv("Company X - Pincode Zones.csv")
invoice_df = pd.read_csv("Courier Company - Invoice.csv")
sku_df = pd.read_csv("Company X - SKU Master.csv")
rate_df = pd.read_csv("Courier Company - Rates.csv")

#Calculate order weight from SKU master
order_df = order_df.merge(sku_df, on='SKU', how='left')
order_df['Order Weight (KG)'] = order_df['Order Qty'] * order_df['Weight (g)'] / 1000
order_wt = order_df.groupby('ExternOrderNo')['Order Weight (KG)'].sum().reset_index()

#Get zone between warehouse and customer pincodes
order_zone = order_wt.merge(pincode_df, left_on='ExternOrderNo', right_on='Customer Pincode', how='left')

In [75]:
print(invoice_df.columns)

Index(['AWB Code', 'Order ID', 'Charged Weight', 'Warehouse Pincode',
       'Customer Pincode', 'Zone', 'Type of Shipment', 'Billing Amount (Rs.)'],
      dtype='object')


In [79]:
result_df = invoice_df.merge(order_zone, left_index=True, right_index=True, how='left')

In [80]:
print(result_df.columns)

Index(['AWB Code', 'Order ID', 'Charged Weight', 'Warehouse Pincode_x',
       'Customer Pincode_x', 'Zone_x', 'Type of Shipment',
       'Billing Amount (Rs.)', 'ExternOrderNo', 'Order Weight (KG)',
       'Warehouse Pincode_y', 'Customer Pincode_y', 'Zone_y'],
      dtype='object')


In [81]:
invoice_df['Order_ID'] = invoice_df['Order ID']
order_zone['Order_ID'] = order_zone['ExternOrderNo']

result_df = invoice_df.merge(order_zone, on='Order_ID', how='left')

In [82]:
#Join Order data with Invoice details
invoice_df['Order_ID'] = invoice_df['Order ID']
order_zone['Order_ID'] = order_zone['ExternOrderNo']

result_df = invoice_df.merge(order_zone, on='Order_ID', how='left')

In [83]:
result_df

,AWB Code,Order ID,Charged Weight,Warehouse Pincode_x,Customer Pincode_x,Zone_x,Type of Shipment,Billing Amount (Rs.),Order_ID,ExternOrderNo,Order Weight (KG),Warehouse Pincode_y,Customer Pincode_y,Zone_y
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,2001806232,2001806232,95.944,NaN,NaN,NaN
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,2001806273,2001806273,47.972,NaN,NaN,NaN
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,2001806408,2001806408,143.916,NaN,NaN,NaN
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,2001806458,2001806458,23.986,NaN,NaN,NaN
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,2001807012,2001807012,11.993,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2,2001812941,2001812941,35.979,NaN,NaN,NaN
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7,2001809383,2001809383,35.979,NaN,NaN,NaN
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4,2001820978,2001820978,23.986,NaN,NaN,NaN
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0,2001811475,2001811475,47.972,NaN,NaN,NaN


In [84]:
order_zone = order_wt.merge(pincode_df, left_on='ExternOrderNo', right_on='Customer Pincode', how='left')

In [86]:
print(order_wt.columns)
print(pincode_df.columns)


Index(['ExternOrderNo', 'Order Weight (KG)'], dtype='object')
Index(['Warehouse Pincode', 'Customer Pincode', 'Zone'], dtype='object')


In [93]:
# Rename columns in order_zone DataFrame to avoid suffixes
order_zone.rename(columns={'Zone': 'Zone_order'}, inplace=True)

# Join Order data with Invoice details
result_df = invoice_df.merge(order_zone, on='Order ID', how='left')

print(result_df.columns)


Index(['AWB Code', 'Order ID', 'Charged Weight', 'Warehouse Pincode_x',
       'Customer Pincode_x', 'Zone', 'Type of Shipment',
       'Billing Amount (Rs.)', 'Order_ID', 'Order Weight (KG)',
       'Warehouse Pincode_y', 'Customer Pincode_y', 'Zone_order'],
      dtype='object')


In [97]:
# Calculate expected shipment weight slab and charges for Company X
def get_weight_slab(row):
    if row['Zone_order'] == 'a':
        return round(row['Order Weight (KG)'] * 4) / 4
    elif row['Zone_order'] == 'b':
        return round(row['Order Weight (KG)'] * 2) / 2
    elif row['Zone_order'] == 'c':
        return round(row['Order Weight (KG)'] * 1.25) / 1.25
    elif row['Zone_order'] == 'd':
        return round(row['Order Weight (KG)'] * 1.5) / 1.5
    else:
        return round(row['Order Weight (KG)'] * 2) / 2

def get_exp_charges(row):
    zone = row['Zone_order']
    weight_slab = row['Order Weight (KG)']

    # Filterring  based on zone and weight slab
    filtered_rates = rate_df[(rate_df['Zone'] == zone) & (rate_df['Weight Slabs'] == weight_slab)]

    if not filtered_rates.empty:
        fwd_fix = filtered_rates['Forward Fixed Charge'].values[0]
        add_rate = filtered_rates['Forward Additional Weight Slab Charge'].values[0]

        slab_len = 0.25 if zone == 'a' else 0.5 if zone == 'b' else 0.75 if zone == 'c' else 1.5 if zone == 'd' else 2
        add_wt = max(row['Order Weight (KG)'] - slab_len, 0)
        add_chg = add_wt * add_rate

        exp_chg = fwd_fix + add_chg
    else:

        exp_chg = 0

    return exp_chg

# Function to create a new column 'Expected Charge' in result_df
result_df['Expected Charge (Rs.)'] = result_df.apply(get_exp_charges, axis=1)

# Functions to create new columns in result_df
result_df['Order Weight Slab (KG)'] = result_df.apply(get_weight_slab, axis=1)
result_df['Expected Charge (Rs.)'] = result_df.apply(get_exp_charges, axis=1)


In [103]:
#Comparing expected vs actual charges
result_df['Diff in Charges (Rs.)'] = result_df['Expected Charge (Rs.)'] - result_df['Billing Amount (Rs.)']

# Order level calculation
order_level_calculation = result_df[['Order ID', 'AWB Code', 'Order Weight (KG)', 'Order Weight Slab (KG)',
                                     'Charged Weight', 'Zone', 'Zone_order', 'Expected Charge (Rs.)',
                                     'Billing Amount (Rs.)', 'Diff in Charges (Rs.)']]

# Summary table
correct_count = (result_df['Diff in Charges (Rs.)'] == 0).sum()
correct_amount = result_df.loc[result_df['Diff in Charges (Rs.)'] == 0, 'Billing Amount (Rs.)'].sum()
over_count = (result_df['Diff in Charges (Rs.)'] < 0).sum()
over_amount = -(result_df.loc[result_df['Diff in Charges (Rs.)'] < 0, 'Diff in Charges (Rs.)'].sum())
under_count = (result_df['Diff in Charges (Rs.)'] > 0).sum()
under_amount = result_df.loc[result_df['Diff in Charges (Rs.)'] > 0, 'Diff in Charges (Rs.)'].sum()

c = pd.DataFrame({'Count': [correct_count, over_count, under_count],
                              'Amount (Rs.)': [correct_amount, over_amount, under_amount]},
                             index=['Total orders where X has been correctly charged',
                                    'Total Orders where X has been overcharged',
                                    'Total Orders where X has been undercharged'])
print(summary_table)


                                                 Count  Amount (Rs.)
Total orders where X has been correctly charged      0           0.0
Total Orders where X has been overcharged          124       13648.2
Total Orders where X has been undercharged           0           0.0


In [99]:
# Save order_level_calculationto CSV
order_level_calculation.to_csv('order_level_calculation.csv', index=False)

# Save summary_table to CSV
summary_table.to_csv('summary_table.csv', index=False)
